In [2]:
#pip install genderdecoder3

  Preparing metadata (setup.py) ... done
  Created wheel for genderdecoder3: filename=genderdecoder3-0.5-py3-none-any.whl size=2999 sha256=96fd18bd04f33f984880d1f553dd42edf234c22715024af70f97bd862ae8173c
  Stored in directory: /Users/yuleumpark/Library/Caches/pip/wheels/aa/1a/ef/87d8e87211ab3a1c7b28cc260b268d32763ef36b66ba79c753
Successfully built genderdecoder3

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import genderdecoder3
import re
import pandas as pd
from collections import Counter
from genderdecoder3 import assess
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
real_df = pd.read_csv("labelled_cleaned.csv")

In [3]:
real_df.head()

,company_name,job_description
0,ministerie van financi n: belastingdienst,tmap next foundation ervaring met projecten in...
1,belastingdienst,"- gecertificeerd scrum master, minimaal psm ii..."
2,duo,3 jaar ervaring in het volgende eisenpakket:\n...
3,dienst uitvoering onderwijs (duo),5 jaar ervaring met de volgende eisen:\n relev...
4,asml netherlands b.v.,location: veldhoven\nnone\n company info: \n ...


In [4]:
job_descriptions = real_df['job_description'].astype(str).tolist()
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(job_descriptions)
feature_names = vectorizer.get_feature_names_out()

def tfidf_bias_score(tfidf_vector, feature_names, masc_words, fem_words):
	tfidf_scores = dict(zip(feature_names, tfidf_vector.toarray()[0]))
	masc_score = sum(tfidf_scores.get(word, 0) for word in masc_words)
	fem_score = sum(tfidf_scores.get(word, 0) for word in fem_words)
	return masc_score - fem_score

# Apply decoding and scoring per job
bias_scores = []
gender_labels = []
masc_words = []
fem_words = []

for i, text in enumerate(job_descriptions):
	result = assess(text)  # returns a dict directly
	masc_words += result['masculine_coded_words']
	fem_words += result['feminine_coded_words']
	bias_score = tfidf_bias_score(tfidf_matrix[i], feature_names, masc_words, fem_words)

	bias_scores.append(bias_score)
	gender_labels.append(result['result'])

def classify_coding(masc_words, fem_words):
	if len(masc_words) > len(fem_words):
		return "masculine-coded", (len(masc_words) - len(fem_words))
	elif len(fem_words) > len(masc_words):
		return "feminine-coded", (len(masc_words) - len(fem_words))
	else:
		return "neutral", (len(masc_words) - len(fem_words))

In [5]:
feminine_coded_words = [
    "agree", "affectionate", "child", "cheer", "collab", "commit", "communal",
    "compassion", "connect", "considerate", "cooperat", "co-operat", "depend",
    "emotiona", "empath", "feel", "flatterable", "gentle", "honest",
    "interpersonal", "interdependen", "interpersona", "inter-personal", "inter-dependen",
    "inter-persona", "kind", "kinship", "loyal", "modesty", "nag", "nurtur",
    "pleasant", "polite", "quiet", "respon", "sensitiv", "submissive", "support",
    "sympath", "tender", "together", "trust", "understand", "warm", "whin",
    "enthusias", "inclusive", "yield", "share", "sharin"
]

masculine_coded_words = [
    "active", "adventurous", "aggress", "ambitio", "analy", "assert", "athlet",
    "autonom", "battle", "boast", "challeng", "champion", "compet", "confident",
    "courag", "decid", "decision", "decisive", "defend", "determin", "domina",
    "dominant", "driven", "fearless", "fight", "force", "greedy", "head-strong",
    "headstrong", "hierarch", "hostil", "impulsive", "independen", "individual",
    "intellect", "lead", "logic", "objective", "opinion", "outspoken", "persist",
    "principle", "reckless", "self-confiden", "self-relian", "self-sufficien",
    "selfconfiden", "selfrelian", "selfsufficien", "stubborn", "superior", "unreasonab"
]

In [9]:
from deep_translator import GoogleTranslator
import re

def translate_stems(stems):
	translator = GoogleTranslator(source="en", target="nl")
	translated = []
	for stem in stems:
		try:
			translated.append(translator.translate(stem))
		except:
			translated.append(stem)
	return translated

translated_feminine = translate_stems(feminine_coded_words)
translated_masculine = translate_stems(masculine_coded_words)


In [11]:
masculine_coded_words.append(translated_masculine)
feminine_coded_words.append(translated_feminine)

In [ ]:
for word in translated_feminine:
	print(f'{word}\n')

In [12]:
# Count partial matches
def count_partial_matches(tokens, stem_list):
    return sum(1 for token in tokens for stem in stem_list if stem in token)

# Main loop: analyze each job description
for idx, row in synthetic_df.iterrows():
    job_description_nl = str(row["job_description"])
    tokens = re.findall(r'\b\w+\b', job_description_nl.lower())

    fem_count = count_partial_matches(tokens, feminine_coded_words)
    masc_count = count_partial_matches(tokens, masculine_coded_words)

    # Save counts
    synthetic_df.at[idx, "feminine_word_count"] = fem_count
    synthetic_df.at[idx, "masculine_word_count"] = masc_count

    # Assign bias label
    if masc_count > fem_count:
        label = "Mannelijke bias"
    elif fem_count > masc_count:
        label = "Vrouwelijke bias"
    else:
        label = "Neutraal"
    synthetic_df.at[idx, "bias_label"] = label

# Print final result
print(synthetic_df[["job_title", "feminine_word_count", "masculine_word_count", "bias_label"]])

NameError: name 'synthetic_df' is not defined